In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('imdb_dataset.csv')

In [ ]:
# Binary Classification is apparent.

df.sample(7)

,review,sentiment
48201,Nothing short of magnificent photography/cinem...,positive
41369,"The worst film ever made, bar none. Give yours...",negative
4447,I'm going to write about this movie and about ...,negative
4789,"John Cassavetes' ""Opening Night"" is fantastic ...",positive
33053,The one line summary is actually the punch lin...,negative
19875,I'll admit that this isn't a great film. It pr...,negative
42065,One Night at McCool's is a very funny movie th...,positive


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk


In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VICTUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VICTUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VICTUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
def preprocess_text(text):
    text = text.lower()
    
    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)
    
    # Remove special characters and numbers
    text = re.sub('[^a-zA-Z\s]', '', text)

    tokens = word_tokenize(text)
    
    # Remove stopwords (They're not relevant)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [10]:
df['processed_review'] = df['review'].apply(preprocess_text)

In [11]:
df.sample(7)

,review,sentiment,processed_review
11541,Just watched this movie and it's not bad; ther...,positive,watched movie bad tense moment lot long dialog...
1562,"I play final fantasy 7, and this movie is EXCE...",positive,play final fantasy movie excellent like game f...
15623,"This is the epitome of bad 80's film-making, u...",negative,epitome bad filmmaking unless prepubescent gir...
34970,Kurt Russell (as Steven Post) works as a mail-...,negative,kurt russell steven post work mailboy struggli...
15583,Most of the silent films I've seen have been s...,positive,silent film ive seen serious nature fun see on...
1341,How Disney can you get? Preppy rich girls act ...,negative,disney get preppy rich girl act like idiot buy...
47192,I have been feeling a little disappointed by T...,positive,feeling little disappointed tarantino ever sin...


In [12]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\VICTUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VICTUS\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [13]:
df['bert_features'] = df['processed_review'].apply(get_bert_embedding)

KeyboardInterrupt: 